In [5]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from collections import OrderedDict

In [2]:
df = pd.read_csv('submissions.csv')
df.head()

,username,ID,contestId,index,OK,WRONG_ANSWER,TIME_LIMIT_EXCEEDED,COMPILATION_ERROR,RUNTIME_ERROR,IDLENESS_LIMIT_EXCEEDED,...,TESTING,PRESENTATION_ERROR,FAILED,CRASHED,CHALLENGED,REJECTED,PARTIAL,SECURITY_VIOLATED,INPUT_PREPARATION_CRASHED,SUBMITTED
0,akshaygahlot73,131689,1646,D,1,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,akshaygahlot73,129692,1621,E,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,akshaygahlot73,129689,1621,D,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,akshaygahlot73,129052,1613,E,1,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,akshaygahlot73,129130,1614,D1,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.shape

(4866956, 22)

In [3]:
df.drop(['contestId', 'index', 'COMPILATION_ERROR', 'SKIPPED', 'TESTING','PRESENTATION_ERROR','FAILED','CRASHED','CHALLENGED','REJECTED','PARTIAL','SECURITY_VIOLATED','INPUT_PREPARATION_CRASHED','SUBMITTED'], axis = 1, inplace=True)
df.head()

,username,ID,OK,WRONG_ANSWER,TIME_LIMIT_EXCEEDED,RUNTIME_ERROR,IDLENESS_LIMIT_EXCEEDED,MEMORY_LIMIT_EXCEEDED
0,akshaygahlot73,131689,1,3,0,0,0,0
1,akshaygahlot73,129692,2,0,0,0,0,0
2,akshaygahlot73,129689,1,2,0,0,0,0
3,akshaygahlot73,129052,1,2,1,0,0,0
4,akshaygahlot73,129130,1,2,0,0,0,0


In [64]:
users = df['username'].unique()

In [65]:
users

array(['akshaygahlot73', 't.g.g.t', 'microtony', ..., 'black_red',
       'naresh6436', 'anonymous151'], dtype=object)

In [17]:
db = pd.DataFrame()

In [16]:
db.append(df[df['username'] == 'user1'])

,username,ID,OK,WRONG_ANSWER,TIME_LIMIT_EXCEEDED,RUNTIME_ERROR,IDLENESS_LIMIT_EXCEEDED,MEMORY_LIMIT_EXCEEDED


In [10]:
def mean(WA, RE, TLE, cnt):
    return (WA + 0.6 * TLE + 0.8 * RE) / max(1, cnt)
    
def rating(OK, WA, RE, TLE, avg):
    ans = 0
    ans = abs(OK * avg - WA - 0.6 * TLE - 0.8 * RE)
    return ans

### Finding average number of Wrong attempts, TLE submissions, etc

In [8]:
# user = users[0]
# WA = [0]
# TLE = [0]
# RE = [0]
# cnt = 0
# j = 0
# means = []
# for i in range(df.shape[0]):
#     if (df.at[i, 'username'] != user):
        
#         means.append(mean(WA, RE, TLE, cnt))
#         # print(f'User {user} has been processed')
#         j += 1
#         user = df.at[i, 'username']
#         WA = RE = TLE = cnt = 0
#         continue
#     WA += (df.at[i, 'WRONG_ANSWER'] + df.at[i, 'MEMORY_LIMIT_EXCEEDED'])
#     TLE += (df.at[i, 'TIME_LIMIT_EXCEEDED'] + df.at[i, 'IDLENESS_LIMIT_EXCEEDED'])
#     RE += df.at[i, 'RUNTIME_ERROR']
#     cnt += 1

In [74]:
cnt = 0 
idx = 0
for i in users:
    db = df[df['username'] == i]
    m1 = np.mean(db['WRONG_ANSWER'] + db['MEMORY_LIMIT_EXCEEDED'])
    m2 = np.mean(db['TIME_LIMIT_EXCEEDED'] + db['IDLENESS_LIMIT_EXCEEDED'])
    m3 = np.mean(db['RUNTIME_ERROR'])
    print(idx, f'{i} has been processed')
    idx += 1
    m = m1 + 0.6 * m2 + 0.8 * m3
    for j, row in db.iterrows():
        db.at[j, 'rating'] = rating(db.at[j, 'OK'], db.at[j, 'WRONG_ANSWER'] + db.at[j, 'MEMORY_LIMIT_EXCEEDED'], db.at[j, 'RUNTIME_ERROR'], db.at[j, 'TIME_LIMIT_EXCEEDED'] + db.at[j, 'IDLENESS_LIMIT_EXCEEDED'], m)
    mean_, sd = np.mean(db['rating']), np.std(db['rating'])
    db.loc[:, 'rating'] = (db['rating'] - mean_) / sd
    df[df['username'] == i]['rating'] = db['rating']
    cnt += db.shape[0]
df.loc[:, 'rating'] = df['rating'].apply(lambda x: abs(x) if (x < 1 and x > -1) else 1)
# print(m1, m2, m3)

0 akshaygahlot73 has been processed


/Users/uttammittal02/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


1 t.g.g.t has been processed
2 microtony has been processed
3 1302kong has been processed
4 Elk-Cloner has been processed
5 Prabhi has been processed
6 Jael has been processed
7 GooZy has been processed
8 psdudeman39 has been processed
9 joker23 has been processed
10 rusty_mayank has been processed
11 contests_only has been processed
12 rakeshhotker has been processed
13 sahaun has been processed
14 kopwer has been processed
15 tcbpg has been processed
16 taxicoo has been processed
17 escepta has been processed
18 guptahimalay99 has been processed
19 20_sarthak has been processed
20 rkelsey has been processed
21 Teshnizi has been processed
22 bin.Go has been processed
23 kunaajain has been processed
24 shawnoodle has been processed
25 I_Need_Alex_R_Coleman has been processed
26 aloksharma has been processed
27 lupvasile has been processed
28 nguyenchicuong has been processed
29 I_hate_adhoc_problems has been processed
30 zerokugi has been processed
31 ffao has been processed
32 JohnPau

KeyboardInterrupt: 

In [97]:
dx = (df[df['username'] == 'microtony'].index)

In [99]:
for i in dx:
    print(i)

1445
1446
1447
1448
1449
1450
1451
1452
1453
1454
1455
1456
1457
1458
1459
1460
1461
1462
1463
1464
1465
1466
1467
1468
1469
1470
1471
1472
1473
1474
1475
1476
1477
1478
1479
1480
1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491
1492
1493
1494
1495
1496
1497
1498
1499
1500
1501
1502
1503
1504
1505
1506
1507
1508
1509
1510
1511
1512
1513
1514
1515
1516
1517
1518
1519
1520
1521
1522
1523
1524
1525
1526
1527
1528
1529
1530
1531
1532
1533
1534
1535
1536
1537
1538
1539
1540
1541
1542
1543
1544
1545
1546
1547
1548
1549
1550
1551
1552
1553
1554
1555
1556
1557
1558
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569
1570
1571
1572
1573
1574
1575
1576
1577
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1589
1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600
1601
1602
1603
1604
1605
1606
1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638
1639
1640
1641
1642
1643
1644


In [85]:
db['rating']

324093   -0.137184
324094   -0.954192
324095   -0.137184
324096   -0.137184
324097   -0.137184
            ...   
325844   -0.954192
325845   -0.137184
325846    0.898608
325847    0.971535
325848   -0.137184
Name: rating, Length: 1756, dtype: float64

In [89]:
df[df['username'] == 'rick_bai']

,username,ID,OK,WRONG_ANSWER,TIME_LIMIT_EXCEEDED,RUNTIME_ERROR,IDLENESS_LIMIT_EXCEEDED,MEMORY_LIMIT_EXCEEDED,rating
324093,rick_bai,118006,1,0,0,0,0,0,NaN
324094,rick_bai,107843,1,1,0,0,0,0,NaN
324095,rick_bai,122083,1,0,0,0,0,0,NaN
324096,rick_bai,33043,1,0,0,0,0,0,NaN
324097,rick_bai,114021,1,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...
325844,rick_bai,93520,1,1,0,0,0,0,NaN
325845,rick_bai,93920,1,0,0,0,0,0,NaN
325846,rick_bai,100413,0,2,0,0,0,0,NaN
325847,rick_bai,100400,1,3,0,0,0,0,NaN


In [76]:
db.loc[:, 'user_id'] = 1034

/Users/uttammittal02/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [79]:
db

,username,ID,OK,WRONG_ANSWER,TIME_LIMIT_EXCEEDED,RUNTIME_ERROR,IDLENESS_LIMIT_EXCEEDED,MEMORY_LIMIT_EXCEEDED,rating,user_id
324093,rick_bai,118006,1,0,0,0,0,0,-0.137184,1034
324094,rick_bai,107843,1,1,0,0,0,0,-0.954192,1034
324095,rick_bai,122083,1,0,0,0,0,0,-0.137184,1034
324096,rick_bai,33043,1,0,0,0,0,0,-0.137184,1034
324097,rick_bai,114021,1,0,0,0,0,0,-0.137184,1034
...,...,...,...,...,...,...,...,...,...,...
325844,rick_bai,93520,1,1,0,0,0,0,-0.954192,1034
325845,rick_bai,93920,1,0,0,0,0,0,-0.137184,1034
325846,rick_bai,100413,0,2,0,0,0,0,0.898608,1034
325847,rick_bai,100400,1,3,0,0,0,0,0.971535,1034


In [73]:
for index, row in db.iterrows():
    print(index)
    print(row)

1445
username                   microtony
ID                            120424
OK                                 0
WRONG_ANSWER                       4
TIME_LIMIT_EXCEEDED                0
RUNTIME_ERROR                      0
IDLENESS_LIMIT_EXCEEDED            0
MEMORY_LIMIT_EXCEEDED              0
rating                           NaN
Name: 1445, dtype: object
1446
username                   microtony
ID                            120403
OK                                 0
WRONG_ANSWER                       1
TIME_LIMIT_EXCEEDED                0
RUNTIME_ERROR                      0
IDLENESS_LIMIT_EXCEEDED            0
MEMORY_LIMIT_EXCEEDED              0
rating                           1.0
Name: 1446, dtype: object
1447
username                   microtony
ID                            120400
OK                                 1
WRONG_ANSWER                       0
TIME_LIMIT_EXCEEDED                0
RUNTIME_ERROR                      0
IDLENESS_LIMIT_EXCEEDED            0
MEMORY_L

In [70]:
db = df[df['username'] == 'microtony']

In [71]:
db

,username,ID,OK,WRONG_ANSWER,TIME_LIMIT_EXCEEDED,RUNTIME_ERROR,IDLENESS_LIMIT_EXCEEDED,MEMORY_LIMIT_EXCEEDED,rating
1445,microtony,120424,0,4,0,0,0,0,NaN
1446,microtony,120403,0,1,0,0,0,0,1.0
1447,microtony,120400,1,0,0,0,0,0,1.0
1448,microtony,117292,0,6,7,0,0,0,1.0
1449,microtony,117289,1,2,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...
1112442,microtony,20403,1,0,0,0,0,0,1.0
1112443,microtony,20400,1,0,0,0,0,0,1.0
1112444,microtony,20326,1,3,0,0,0,0,1.0
1112445,microtony,20323,1,0,0,0,0,0,1.0


In [18]:
db['rating'] -= db['rating'].mean()

/var/folders/xj/gnsh9k6900j06gz7jh26w2b40000gn/T/ipykernel_85846/3776080362.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['rating'] -= db['rating'].mean()


In [22]:
db['rating']

0       1.435377
1       3.129247
2       0.435377
3       1.035377
4       0.435377
          ...   
1137    0.564623
1138    6.435377
1139    0.435377
1140    1.564623
1141    1.564623
Name: rating, Length: 1142, dtype: float64

In [42]:
mean_ = db['rating'].mean()
sd = db['rating'].std()
mean_, sd

(1.9174606874595548, 2.1394552084194722)

In [44]:
db.loc[:, 'rating'] /= sd

/Users/uttammittal02/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [47]:
db['rating']

0       0.225330
1       0.566399
2       0.692739
3       0.412294
4       0.692739
          ...   
1137    0.632328
1138    2.111713
1139    0.692739
1140    0.164919
1141    0.164919
Name: rating, Length: 1142, dtype: float64

In [52]:
db['rating'].mean()

0.5328645486231219

In [46]:
db.loc[:, 'rating'] = db['rating'].apply(lambda x: x if x > 0 else -x)

In [35]:
a = np.array(db['rating'])
a

array([0.48208416, 1.21178625, 1.48208416, ..., 1.48208416, 0.35283722,
       0.35283722])

In [62]:
dl = db[db['rating'] > 0.5]
# dl = dl[dl['rating'] < 1]

In [63]:
dl['rating']

1       0.566648
2       0.693043
4       0.693043
7       1.000000
8       0.693043
          ...   
1132    1.000000
1135    1.000000
1137    0.632605
1138    1.000000
1139    0.693043
Name: rating, Length: 455, dtype: float64

In [20]:
db['rating']

0      -0.482084
1       1.211786
2      -1.482084
3      -0.882084
4      -1.482084
          ...   
1137   -1.352837
1138    4.517916
1139   -1.482084
1140   -0.352837
1141   -0.352837
Name: rating, Length: 1142, dtype: float64

In [4]:
db = df[df['username'] == 'akshaygahlot73']
db.head()

,username,ID,OK,WRONG_ANSWER,TIME_LIMIT_EXCEEDED,RUNTIME_ERROR,IDLENESS_LIMIT_EXCEEDED,MEMORY_LIMIT_EXCEEDED
0,akshaygahlot73,131689,1,3,0,0,0,0
1,akshaygahlot73,129692,2,0,0,0,0,0
2,akshaygahlot73,129689,1,2,0,0,0,0
3,akshaygahlot73,129052,1,2,1,0,0,0
4,akshaygahlot73,129130,1,2,0,0,0,0


In [9]:
means

[array([1.56462347]),
 1.223841059602649,
 0.8865900383141763,
 1.1866666666666668,
 1.587281399046105,
 0.3826589595375723,
 1.020773313115997,
 1.0492569002123142,
 0.8171689497716894,
 1.2906250000000001,
 1.0203821656050958,
 0.8486792452830187,
 0.6930232558139535,
 0.9018912529550828,
 0.9309401709401708,
 1.5629139072847682,
 1.3726269315673292,
 1.3865257595772786,
 1.1214953271028036,
 0.8736728060671723,
 1.4296296296296296,
 1.4560509554140126,
 1.228828828828829,
 0.7907473309608541,
 1.1586206896551725,
 1.5278350515463917,
 0.7094147582697201,
 1.4000000000000001,
 1.30020202020202,
 1.2096,
 1.1773519163763067,
 0.8351464435146443,
 0.8783821478382147,
 1.3655058043117747,
 0.783157894736842,
 0.95390625,
 1.1988321167883211,
 1.6452285485164393,
 0.8808429118773946,
 0.9885985748218526,
 0.771957671957672,
 0.7210526315789474,
 1.352567237163814,
 3.2252100840336135,
 0.9498054474708171,
 1.3311284046692609,
 1.1472190257000383,
 1.1885892116182573,
 1.6864321608040203,

In [ ]:
j = 0
maxima = []
t = np.empty([df.shape[0], 1],dtype= OrderedDict)
_ = 0
user = users[0]
for i in range(df.shape[0]):
    d = OrderedDict()
    d['user'] = df.at[i, 'username']
    d['problem'] = df.at[i, 'ID']
    if (user != df.at[i, 'username']):
        j += 1
        print(j, 'users has been processsed.')
        user = df.at[i, 'username']
        maxima.append(_)
        _ = 0
    d['rating'] = rating(df.at[i, 'OK'], df.at[i, 'WRONG_ANSWER'] + df.at[i, 'MEMORY_LIMIT_EXCEEDED'], df.at[i, 'RUNTIME_ERROR'], df.at[i, 'TIME_LIMIT_EXCEEDED'] + df.at[i, 'IDLENESS_LIMIT_EXCEEDED'], means[j])
    if _ > d['rating'] : 
      _ = d['rating']
    t[i] = d
    # a = len(t)
#     print(i, 'entries processed.')
print(t)

In [100]:
db = pd.DataFrame(t)

In [ ]:
user_mapping = zip(users, maxima)

In [ ]:
for i in range(len(t)):
    t[i]['rating'] /= user_mapping[t[i]['user']]

In [86]:
print(i)

4059720


In [87]:
print(len(t))

0


In [88]:
print(t)

[]


In [89]:
a = np.array([1, 2, 3])

In [95]:
a = np.append(a, [4])

In [96]:
a

array([1, 2, 3, 4])